In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score

In [ ]:
# Carregar dados
dados = pd.read_csv('a1-in.csv', sep=',')
dados.head()


,UNIQUEID,SCHOOL,Class,GRADE,CODER,STUDENTID,Gender,OBSNUM,totalobs-forsession,Activity,ONTASK,TRANSITIONS,NumACTIVITIES,FORMATchanges,NumFORMATS,Obsv/act,Transitions/Durations,Total Time
0,14400,B,T9Q,0,Z,600865,0,1,0,Wholecarpet,Y,3,4,1,2,770.5,0.004043,0
1,14401,B,T9Q,0,Z,596466,0,1,1,Wholecarpet,Y,3,4,1,2,770.5,0.004043,23
2,14402,B,T9Q,0,Z,616590,0,1,2,Wholecarpet,Y,3,4,1,2,770.5,0.004043,25
3,14403,B,T9Q,0,Z,734358,1,1,3,Wholecarpet,Y,3,4,1,2,770.5,0.004043,27
4,14404,B,T9Q,0,Z,826308,1,1,4,Wholecarpet,Y,3,4,1,2,770.5,0.004043,31


Identificação das Variáveis Relevantes:

Gender: Representa o gênero do participante.
Activity: Refere-se à atividade realizada pelo participante.
TRANSITIONS: Número de transições entre diferentes atividades.
NumACTIVITIES: Número total de atividades realizadas.
FORMATchanges: Número de mudanças de formato durante as atividades.
NumFORMATS: Número total de formatos diferentes utilizados durante as atividades.
ONTASK: Variável alvo que indica se o participante permaneceu focado na tarefa.

Para determinar quais variáveis fazem sentido para a generalização do modelo, precisamos considerar aquelas que têm relevância teórica e prática para prever a variável alvo. No contexto fornecido, onde estamos tentando prever se um participante permaneceu focado em uma tarefa durante uma atividade, as seguintes variáveis podem fazer sentido:

Gênero: Pode haver diferenças de comportamento entre os gêneros que influenciam a capacidade de permanecer focado em uma tarefa.
Atividade: O tipo de atividade realizada pode afetar a capacidade de concentração do participante. Por exemplo, atividades mais desafiadoras podem exigir mais atenção.
Número de transições entre atividades: Um alto número de transições pode indicar falta de foco ou distração.
Número total de atividades realizadas: Pode haver uma relação entre a quantidade de atividades realizadas e a capacidade de manter o foco ao longo do tempo.
Número de mudanças de formato durante as atividades: Mudanças frequentes de formato podem indicar distração ou falta de engajamento.
Número total de formatos diferentes utilizados durante as atividades: Pode haver uma relação entre a variedade de formatos utilizados e a capacidade de concentração do participante.
Variável alvo (ONTASK): Esta é a variável que estamos tentando prever e, portanto, é crucial incluí-la como parte do conjunto de dados.
Portanto, essas variáveis foram selecionadas como relevantes para a construção do modelo preditivo. Elas são fundamentais para entender e prever o comportamento dos participantes durante as atividades, pois capturam diferentes aspectos que podem influenciar sua capacidade de permanecer focado na tarefa.

Separação de todos os dados em conjunto de treinamento e teste

In [ ]:
# Variáveis relevantes
X = dados[['Gender', 'Activity', 'TRANSITIONS', 'NumACTIVITIES', 'FORMATchanges']]
#Variavel de saida
y = dados['ONTASK']

In [ ]:
# Separação de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Definir as transformações para características numéricas e categóricas
numeric_features = ['TRANSITIONS', 'NumACTIVITIES', 'FORMATchanges']
categorical_features = ['Gender', 'Activity']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)])


Pré-processamento dos Dados com pipelines

In [ ]:
# Configurar os pipelines para cada algoritmo
pipelines = [
    ('Logistic Regression', Pipeline([('preprocessor', preprocessor), ('classifier', LogisticRegression())])),
    ('Decision Tree', Pipeline([('preprocessor', preprocessor), ('classifier', DecisionTreeClassifier())])),
   # ('SVM', Pipeline([('preprocessor', preprocessor), ('classifier', SVC())]))
]

Configura a hiperparametrizacao e apresenta os dados

In [ ]:
# Configurar hiperparâmetros
hyperparameters = {
    'Logistic Regression': {
        'classifier__C': np.logspace(-3, 3, 7),
        'classifier__penalty': ['l1', 'l2'],
        'classifier__solver': ['liblinear']
    },
    'Decision Tree': {
        'classifier__max_depth': range(1, 11),
        'classifier__min_samples_split': range(2, 11)
    },
    'SVM': {
       'classifier__C': np.logspace(-2, 2, 5),
       'classifier__kernel': ['linear', 'rbf']
    }
}

In [ ]:
# Treinar e avaliar os modelos
for name, pipeline in pipelines:
    model = GridSearchCV(pipeline, hyperparameters[name], cv=3, n_jobs=-1)
    model.fit(X_train, y_train)

    # Avaliar desempenho nos dados de treinamento
    train_accuracy = model.score(X_train, y_train)
    train_predictions = model.predict(X_train)
    train_precision = precision_score(y_train, train_predictions, pos_label='Y')
    train_recall = recall_score(y_train, train_predictions, pos_label='Y')
    train_f1 = f1_score(y_train, train_predictions, pos_label='Y')
    train_balanced_accuracy = balanced_accuracy_score(y_train, train_predictions)

    print(f"\nDesempenho de {name} nos dados de treinamento:")
    print(f"Acurácia: {train_accuracy:.2f}")
    print(f"Precisão: {train_precision:.2f}")
    print(f"Recall: {train_recall:.2f}")
    print(f"F1: {train_f1:.2f}")
    print(f"Acurácia Balanceada: {train_balanced_accuracy:.2f}")

    # Avaliar desempenho nos dados de teste
    test_accuracy = model.score(X_test, y_test)
    test_predictions = model.predict(X_test)
    test_precision = precision_score(y_test, test_predictions, pos_label='Y')
    test_recall = recall_score(y_test, test_predictions, pos_label='Y')
    test_f1 = f1_score(y_test, test_predictions, pos_label='Y')
    test_balanced_accuracy = balanced_accuracy_score(y_test, test_predictions)

    print(f"\nDesempenho de {name} nos dados de teste:")
    print(f"Acurácia: {test_accuracy:.2f}")
    print(f"Precisão: {test_precision:.2f}")
    print(f"Recall: {test_recall:.2f}")
    print(f"F1: {test_f1:.2f}")
    print(f"Acurácia Balanceada: {test_balanced_accuracy:.2f}")
    print()


Desempenho de Logistic Regression nos dados de treinamento:
Acurácia: 0.67
Precisão: 0.67
Recall: 1.00
F1: 0.80
Acurácia Balanceada: 0.50

Desempenho de Logistic Regression nos dados de teste:
Acurácia: 0.68
Precisão: 0.68
Recall: 1.00
F1: 0.81
Acurácia Balanceada: 0.50


Desempenho de Decision Tree nos dados de treinamento:
Acurácia: 0.67
Precisão: 0.67
Recall: 1.00
F1: 0.80
Acurácia Balanceada: 0.50

Desempenho de Decision Tree nos dados de teste:
Acurácia: 0.68
Precisão: 0.68
Recall: 1.00
F1: 0.81
Acurácia Balanceada: 0.50



Parece que todos os modelos (Logistic Regression, Decision Tree e SVM) têm desempenhos muito semelhantes nos dados de treinamento e teste. Eles alcançam uma acurácia de aproximadamente 0.68 nos dados de teste, com precisão, recall e F1-score de 0.68, 1.00 e 0.81, respectivamente. A acurácia balanceada permanece em 0.50 para todos os modelos.

O desempenho uniforme entre esses modelos sugere que o conjunto de dados pode ser relativamente simples ou que os modelos não estão sendo adequadamente ajustados. A alta pontuação de recall (1.00) sugere que os modelos estão capturando corretamente todos os casos positivos. No entanto, isso é acompanhado por uma precisão modesta (0.68), o que significa que há uma quantidade significativa de falsos positivos.

Dado o desempenho semelhante entre os modelos, pode ser benéfico explorar outras técnicas de modelagem ou ajustar mais profundamente os hiperparâmetros dos modelos existentes. Por exemplo, técnicas como ensemble learning, como Random Forests ou Gradient Boosting, podem oferecer melhor desempenho. Além disso, a otimização dos hiperparâmetros, como ajuste da regularização nos modelos de regressão logística ou otimização dos parâmetros da árvore de decisão, poderia levar a resultados mais significativos.

Em comparação com os resultados do exercício anterior, parece que os modelos apresentam desempenho semelhante. No entanto, isso sugere que os modelos podem estar no limite de sua capacidade preditiva, e melhorias significativas podem exigir abordagens mais avançadas de modelagem ou pré-processamento de dados mais sofisticado.








In [ ]:
# Criar nova feature: proporção de mudanças de formato em relação ao número total de atividades
dados['Proporcao_FORMATchanges'] = dados['FORMATchanges'] / dados['NumACTIVITIES']

# Verificar as primeiras linhas do conjunto de dados com a nova feature
print(dados.head())

# Separar novamente os dados com a nova feature
X = dados[['Gender', 'Activity', 'TRANSITIONS', 'NumACTIVITIES', 'FORMATchanges', 'Proporcao_FORMATchanges']]
y = dados['ONTASK']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


   UNIQUEID SCHOOL Class  GRADE CODER  STUDENTID  Gender  OBSNUM  \
0     14400      B   T9Q      0     Z     600865       0       1   
1     14401      B   T9Q      0     Z     596466       0       1   
2     14402      B   T9Q      0     Z     616590       0       1   
3     14403      B   T9Q      0     Z     734358       1       1   
4     14404      B   T9Q      0     Z     826308       1       1   

   totalobs-forsession     Activity ONTASK  TRANSITIONS  NumACTIVITIES  \
0                    0  Wholecarpet      Y            3              4   
1                    1  Wholecarpet      Y            3              4   
2                    2  Wholecarpet      Y            3              4   
3                    3  Wholecarpet      Y            3              4   
4                    4  Wholecarpet      Y            3              4   

   FORMATchanges  NumFORMATS  Obsv/act  Transitions/Durations  Total Time  \
0              1           2     770.5               0.004043        

Com essa nova feature, estamos considerando não apenas o número absoluto de mudanças de formato, mas também a proporção dessas mudanças em relação ao número total de atividades realizadas. Isso pode ajudar o modelo a capturar melhor a relação entre a variedade de formatos e a capacidade de concentração do participante.

In [ ]:
# Atualizar as variáveis relevantes com a nova feature
X = dados[['Gender', 'Activity', 'TRANSITIONS', 'NumACTIVITIES', 'FORMATchanges', 'Proporcao_FORMATchanges']]
y = dados['ONTASK']

# Separar os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir o preprocessor com a nova feature
numeric_features = ['TRANSITIONS', 'NumACTIVITIES', 'FORMATchanges', 'Proporcao_FORMATchanges']
categorical_features = ['Gender', 'Activity']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)])

# Configurar os pipelines atualizados para cada algoritmo
pipelines = [
    ('Logistic Regression', Pipeline([('preprocessor', preprocessor), ('classifier', LogisticRegression())])),
    ('Decision Tree', Pipeline([('preprocessor', preprocessor), ('classifier', DecisionTreeClassifier())])),
]

# Configurar hiperparâmetros para a busca
hyperparameters = {
    'Logistic Regression': {
        'classifier__C': np.logspace(-3, 3, 7),
        'classifier__penalty': ['l1', 'l2'],
        'classifier__solver': ['liblinear']
    },
    'Decision Tree': {
        'classifier__max_depth': range(1, 11),
        'classifier__min_samples_split': range(2, 11)
    },
}

# Treinar e avaliar os modelos com a nova feature
for name, pipeline in pipelines:
    model = GridSearchCV(pipeline, hyperparameters[name], cv=3, n_jobs=-1)
    model.fit(X_train, y_train)

    # Avaliar desempenho nos dados de treinamento
    train_accuracy = model.score(X_train, y_train)
    train_predictions = model.predict(X_train)
    train_precision = precision_score(y_train, train_predictions, pos_label='Y')
    train_recall = recall_score(y_train, train_predictions, pos_label='Y')
    train_f1 = f1_score(y_train, train_predictions, pos_label='Y')
    train_balanced_accuracy = balanced_accuracy_score(y_train, train_predictions)

    print(f"\nDesempenho de {name} nos dados de treinamento:")
    print(f"Acurácia: {train_accuracy:.2f}")
    print(f"Precisão: {train_precision:.2f}")
    print(f"Recall: {train_recall:.2f}")
    print(f"F1: {train_f1:.2f}")
    print(f"Acurácia Balanceada: {train_balanced_accuracy:.2f}")

    # Avaliar desempenho nos dados de teste
    test_accuracy = model.score(X_test, y_test)
    test_predictions = model.predict(X_test)
    test_precision = precision_score(y_test, test_predictions, pos_label='Y')
    test_recall = recall_score(y_test, test_predictions, pos_label='Y')
    test_f1 = f1_score(y_test, test_predictions, pos_label='Y')
    test_balanced_accuracy = balanced_accuracy_score(y_test, test_predictions)

    print(f"\nDesempenho de {name} nos dados de teste:")
    print(f"Acurácia: {test_accuracy:.2f}")
    print(f"Precisão: {test_precision:.2f}")
    print(f"Recall: {test_recall:.2f}")
    print(f"F1: {test_f1:.2f}")
    print(f"Acurácia Balanceada: {test_balanced_accuracy:.2f}")
    print()



Desempenho de Logistic Regression nos dados de treinamento:
Acurácia: 0.67
Precisão: 0.67
Recall: 1.00
F1: 0.80
Acurácia Balanceada: 0.50

Desempenho de Logistic Regression nos dados de teste:
Acurácia: 0.68
Precisão: 0.68
Recall: 1.00
F1: 0.81
Acurácia Balanceada: 0.50


Desempenho de Decision Tree nos dados de treinamento:
Acurácia: 0.67
Precisão: 0.67
Recall: 1.00
F1: 0.80
Acurácia Balanceada: 0.50

Desempenho de Decision Tree nos dados de teste:
Acurácia: 0.68
Precisão: 0.68
Recall: 1.00
F1: 0.81
Acurácia Balanceada: 0.50



Os resultados foram semelhantes aos da aula8 (no qual o codigo é representado pelo primeiro teste acima deste) os resultados semelhantes após a adição da nova feature podem ser explicados por algumas razões:

Correlação com features existentes: A nova feature pode estar correlacionada com as features já presentes no conjunto de dados. Se isso ocorrer, a adição da nova feature pode não fornecer informações adicionais significativas para os modelos, resultando em desempenho semelhante.
Redundância de informação: A nova feature pode conter informações semelhantes às features existentes, tornando-a redundante. Nesse caso, os modelos podem não se beneficiar da inclusão da nova feature, pois as informações já estão sendo capturadas por outras variáveis.
Complexidade do problema: O problema em questão pode ser relativamente simples, o que significa que os modelos já estão capturando a maioria das relações presentes nos dados com as features existentes. Nesse cenário, a adição de uma nova feature pode não fornecer ganhos significativos de desempenho.
Modelos não sensíveis à nova feature: Os modelos escolhidos (Regressão Logística e Árvore de Decisão) podem não ser sensíveis à nova feature adicionada. Alguns modelos podem não se beneficiar igualmente de todas as features disponíveis, especialmente se as características adicionadas não forem relevantes para a tarefa de predição.
Limitações dos modelos: Os modelos utilizados podem ter limitações intrínsecas que os impedem de aproveitar ao máximo as informações fornecidas pela nova feature. Por exemplo, a Regressão Logística pode ter dificuldades em lidar com relações não-lineares ou interações complexas entre variáveis.

In [ ]:
# Definir todas as features presentes na base de dados original
all_features_original = ['UNIQUEID', 'SCHOOL', 'Class', 'GRADE', 'CODER', 'STUDENTID', 'Gender', 'OBSNUM', 'totalobs-forsession', 'Activity', 'ONTASK', 'TRANSITIONS', 'NumACTIVITIES', 'FORMATchanges', 'NumFORMATS', 'Obsv/act', 'Transitions/Durations', 'Total Time']

# Adicionar a nova feature criada
all_features_original.append('Proporcao_FORMATchanges')

# Separar os dados em features (X) e target (y)
X = dados[all_features_original]
y = dados['ONTASK']

# Separar os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Definir o preprocessor para as features numéricas e categóricas
numeric_features = ['TRANSITIONS', 'NumACTIVITIES', 'FORMATchanges', 'NumFORMATS', 'Obsv/act', 'Transitions/Durations', 'Total Time', 'Proporcao_FORMATchanges']
categorical_features = ['SCHOOL', 'Class', 'CODER', 'Gender', 'Activity']  # Características categóricas presentes na base de dados original
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)])



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

# Definir os classificadores
classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier()
}

# Iterar sobre os classificadores
for clf_name, clf in classifiers.items():
    # Definir o pipeline com RFE para cada classificador
    pipeline_with_rfe = Pipeline([
        ('preprocessor', preprocessor),
        ('feature_selection', RFE(estimator=clf, n_features_to_select=3)),
        ('classifier', clf)
    ])

    # Treinar o modelo com RFE
    pipeline_with_rfe.fit(X_train, y_train)

    num_features_selected = pipeline_with_rfe.named_steps['feature_selection'].support_.sum()
    print(f"\nNumero de features selecionadas pelo RFE :  {num_features_selected}")

    # Avaliar desempenho nos dados de treinamento
    train_accuracy_rfe = pipeline_with_rfe.score(X_train, y_train)
    train_predictions_rfe = pipeline_with_rfe.predict(X_train)
    train_precision_rfe = precision_score(y_train, train_predictions_rfe, pos_label='Y')
    train_recall_rfe = recall_score(y_train, train_predictions_rfe, pos_label='Y')
    train_f1_rfe = f1_score(y_train, train_predictions_rfe, pos_label='Y')
    train_balanced_accuracy_rfe = balanced_accuracy_score(y_train, train_predictions_rfe)

    print(f"\nDesempenho do modelo com seleção de features (RFE) para {clf_name} nos dados de treinamento:")
    print(f"Acurácia: {train_accuracy_rfe:.2f}")
    print(f"Precisão: {train_precision_rfe:.2f}")
    print(f"Recall: {train_recall_rfe:.2f}")
    print(f"F1: {train_f1_rfe:.2f}")
    print(f"Acurácia Balanceada: {train_balanced_accuracy_rfe:.2f}")

    # Avaliar desempenho nos dados de teste
    test_accuracy_rfe = pipeline_with_rfe.score(X_test, y_test)
    test_predictions_rfe = pipeline_with_rfe.predict(X_test)
    test_precision_rfe = precision_score(y_test, test_predictions_rfe, pos_label='Y')
    test_recall_rfe = recall_score(y_test, test_predictions_rfe, pos_label='Y')
    test_f1_rfe = f1_score(y_test, test_predictions_rfe, pos_label='Y')
    test_balanced_accuracy_rfe = balanced_accuracy_score(y_test, test_predictions_rfe)

    print(f"\nDesempenho do modelo com seleção de features (RFE) para {clf_name} nos dados de teste:")
    print(f"Acurácia: {test_accuracy_rfe:.2f}")
    print(f"Precisão: {test_precision_rfe:.2f}")
    print(f"Recall: {test_recall_rfe:.2f}")
    print(f"F1: {test_f1_rfe:.2f}")
    print(f"Acurácia Balanceada: {test_balanced_accuracy_rfe:.2f}")



Numero de features selecionadas pelo RFE :  3

Desempenho do modelo com seleção de features (RFE) para Logistic Regression nos dados de treinamento:
Acurácia: 0.67
Precisão: 0.68
Recall: 0.97
F1: 0.80
Acurácia Balanceada: 0.52

Desempenho do modelo com seleção de features (RFE) para Logistic Regression nos dados de teste:
Acurácia: 0.68
Precisão: 0.69
Recall: 0.97
F1: 0.80
Acurácia Balanceada: 0.53

Numero de features selecionadas pelo RFE :  3

Desempenho do modelo com seleção de features (RFE) para Decision Tree nos dados de treinamento:
Acurácia: 1.00
Precisão: 1.00
Recall: 1.00
F1: 1.00
Acurácia Balanceada: 1.00

Desempenho do modelo com seleção de features (RFE) para Decision Tree nos dados de teste:
Acurácia: 0.60
Precisão: 0.71
Recall: 0.71
F1: 0.71
Acurácia Balanceada: 0.55


Comparando os resultados com e sem RFE para o modelo de Logistic Regression, observamos que a acurácia permaneceu praticamente a mesma nos dados de treinamento e teste, enquanto a precisão e o F1-score aumentaram ligeiramente nos dados de teste com RFE. No entanto, a acurácia balanceada teve um pequeno aumento tanto nos dados de treinamento quanto nos de teste. Isso sugere que a seleção de features com RFE pode ter ajudado a melhorar a precisão do modelo em classificar corretamente as instâncias da classe positiva ('Y').

Já para o modelo de Decision Tree, observamos uma grande diferença de desempenho com RFE. Nos dados de treinamento, o modelo atingiu uma acurácia de 100% após a seleção de features, com precisão, recall e F1-score também atingindo 100%. No entanto, nos dados de teste, embora a precisão tenha aumentado, a acurácia geral diminuiu significativamente, indicando um possível caso de overfitting nos dados de treinamento. A acurácia balanceada também aumentou ligeiramente nos dados de teste, mas o F1-score se manteve relativamente estável.

Em resumo, enquanto a seleção de features com RFE parece ter contribuído para melhorar o desempenho do modelo de Logistic Regression, os resultados para o modelo de Decision Tree mostram uma melhoria significativa nos dados de treinamento, mas um desempenho inferior nos dados de teste, sugerindo a necessidade de ajuste de hiperparâmetros ou consideração de outras abordagens para evitar overfitting.